In [9]:

# !rm -rf LightGBM
import lightgbm as lgb
# %cd ..
# %cd build
# !cmake -DUSE_GPU=1 ..

In [10]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import numpy as np
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torch.optim as optim
from torch.optim import lr_scheduler


In [11]:

import torchvision.models as models
import torch.nn as nn
import time
# from py_files.dataloader2 import *
import copy

In [12]:
import torch
device = 'cpu'

In [14]:
X = torch.empty(0,4608).to(device)
y = torch.empty(0, dtype=torch.long).to(device)
for i in range(1,5):
  loal = torch.load(open("./model_params/feat_extra"+str(i)+".pth", 'rb'), map_location='cpu')
  X = torch.cat((X, loal), dim = 0)

  loal2 = torch.load(open("./model_params/label"+str(i)+".pth", 'rb'),map_location='cpu')
  y = torch.cat((y, loal2), dim = 0)

X_dev = torch.load(open("./model_params/dev_feat_extra.pth", 'rb'),map_location='cpu')
y_dev = torch.load(open("./model_params/dev_label.pth", 'rb'),map_location='cpu')


In [15]:
print(X[2:4])
print(X.size()[1])
print("*"*50)
print(y[0:5])
print(y.size())
print("*"*50)
print(X_dev.size())
print(y_dev.size())

tensor([[ 0.3502,  0.1545,  0.9300,  ...,  0.0409,  0.0211,  0.0114],
        [ 0.8018,  0.1333,  0.4812,  ..., -0.0092,  0.0235,  0.0047]])
4608
**************************************************
tensor([0, 0, 2, 1, 0])
torch.Size([22400])
**************************************************
torch.Size([800, 4608])
torch.Size([800])


In [8]:
# parameters = {
#     'max_depth': [ 6, 7, 9,10],
#     'num_leaves':[ 60, 80, 90, 100],
# }

# gbm = lgb.LGBMClassifier(objective = 'multiclass',
#                           metric = 'multi_logloss',
#                           max_depth = 6,
#                           num_leaves = 60,
#                           learning_rate = 0.1,
#                           feature_fraction = 0.7,
#                           min_child_samples=21,
#                           min_child_weight=0.001,
#                           bagging_fraction = 1,
#                           bagging_freq = 2,
#                           reg_alpha = 0.001,
#                           reg_lambda = 8,
#                           cat_smooth = 0, 
#                           device = 'gpu',
#                           gpu_platform_id = 0, 
#                           gpu_device_id = 0
#                         )
# gsearch = GridSearchCV(gbm, param_grid=parameters, cv=3)
# gsearch.fit(X, y)
# print('参数的最佳取值:{0}'.format(gsearch.best_params_))
# print('最佳模型得分:{0}'.format(gsearch.best_score_))
# print('最佳模型時間:{0}'.format(gsearch.refit_time_))
# print(gsearch.cv_results_['mean_test_score'])
# print(gsearch.cv_results_['params'])

In [31]:


X = X.cpu()
y = y.cpu()
X_dev = X_dev.cpu()
y_dev = y_dev.cpu()
parameters = {
              # 'device':'gpu',
              # 'max_depth': [ 9],
              # 'num_leaves':[255],

              # 'min_data_in_leaf':[21]],
              # 'min_sum_hessian_in_leaf':[0.001],

              # 'feature_fraction': [0.6, 0.7, 0.8, 0.9,1.0],
              # 'bagging_fraction': [0.7,0.8,0.9,1.0],
              # # # 'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0.001],
              'lambda_l2': [8],

              # 'learning_rate': [0.2],
              # # 'cat_smooth': [10, 15, 20, ]
}

best_lgbm = None
best_acc = 0
f1, f2 = np.meshgrid(parameters['lambda_l1'], parameters['lambda_l2'])
f1 = f1.flat[:]
f2 = f2.flat[:]

for i, j in zip(f1, f2):
  lgbm = lgb.LGBMClassifier(objective = 'muilti_class',
                          metric = 'multi_logloss',
                          max_depth = 9,
                          num_leaves = 255,
                          learning_rate = 0.008,
                          # feature_fraction = 0.9,
                          feature_fraction = 0.7,
                          min_child_samples=21,
                          min_child_weight=0.01,
                          # bagging_fraction = 0.8,
                          bagging_fraction = 1,
                          bagging_freq = 2,
                          reg_alpha = i,
                          reg_lambda = j,
                          cat_smooth = 0, 
                          device = 'cpu',
                          n_estimators = 600,
                          )
  lgbm.fit(X, y)
  y_pred = torch.tensor(lgbm.predict(X))
  acc_train = sum(y_pred == y).item()/22400

  y_dev_pred = torch.tensor(lgbm.predict(X_dev))
  acc = sum(y_dev_pred == y_dev).item()/800
  print(f'{i}, {j}, acc in dev:{acc},  acc in train:{acc_train}')
  if best_acc <= acc:
    best_lgbm = lgbm
    best_acc = acc
  
print(best_lgbm, best_acc) 

0.001, 8, acc in dev:0.73375,  acc in train:0.9789732142857143
LGBMClassifier(bagging_fraction=1, bagging_freq=2, cat_smooth=0, device='cpu',
               feature_fraction=0.7, learning_rate=0.008, max_depth=9,
               metric='multi_logloss', min_child_samples=21, n_estimators=600,
               num_leaves=255, objective='muilti_class', reg_alpha=0.001,
               reg_lambda=8) 0.73375


In [22]:
f1

array(['min_data_in_leaf'], dtype='<U16')

In [96]:
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

dtype = torch.float32 # we will be using float throughout this tutorial

#the steps of preprocessing
p = 0.5
q = 0.5
trans_list = [
        transforms.RandomRotation(degrees = (0,359)),#
        transforms.Resize(280),
        transforms.CenterCrop(224),

        transforms.RandomHorizontalFlip(p),  
        transforms.RandomVerticalFlip(q),

        transforms.ColorJitter(brightness=(0, 3), contrast=(
        0, 3), saturation=(0, 5), hue=(0, 0)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.8, scale=(0.02, 0.02), ratio=(1, 1), value='1234'),
    
]
preprocess = {
    'train': transforms.Compose(trans_list)
}

In [106]:
li = [0,1,2]

In [113]:
li.insert(1,4)

In [114]:
li

[3, 4, 3, 0, 3, 2]

0

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(cm.numpy().astype(np.int32),
                     index = [i for i in ["true_A", "true_B", "true_C"]],
                     columns = [i for i in ["pred_A", "pred_B", "pred_C"]])
plt.figure(figsize = (4, 4))
sn.heatmap(df_cm, annot=True, cmap="BuPu")

In [ ]:
cm.numpy().astype(np.int32)

In [ ]:
測試model1的acc

In [ ]:
print(preds1)